# Replication: Zeighami *et al*, 2019

## Introduction

This notebook attempts to reproduce the following paper (which uses the [PPMI](http://ppmi-info.org) dataset):

<div class="alert alert-block alert-success">
Zeighami, Yashar, et al. <a href=https://doi.org/10.1016/j.nicl.2019.101986>Assessment of a prognostic MRI biomarker in early de novo Parkinson's disease.</a> NeuroImage: Clinical 24 (2019): 101986.
</div>

This study used longitudinal MRI data from 362 patients with de novo PD (i.e., patients not yet taking any medication) and 112 healthy controls. Subjects were split into a main cohort with 3T MRI scans (N=222) and a validation cohort with 1.5T MRI scans (N=140). 

The demographics parameters for the PD patients were as follows (table extracted from the paper):

<img src="images/demographics.png" width=800/>

This paper used a method introduced in [Zeighami *et al* (2015)](https://doi.org/10.7554/eLife.08440) consisting of applying Independent Component Analysis (ICA) to Deformation-Based Morphometry (DBM) maps to identify an atrophy network related to PD. The main result is atrophy measures from this PD network is significantly correlated with disease progression as measured by differences in clinical and/or cognitive scores between the baseline visit and the most recent follow-up visit. This is illustrated in the top row of the following table (extracted from the paper):

<img src="images/results_correlations.png" width=600/>

The authors also showed that the PD-related atrophy score can be used to identify patients who have a 1.5 standard deviation change in the global composite outcome (a measure they defined) between baseline and follow-up visits. The PD-related atrophy biomarker achieved an area under the receiver operating characteristics curve (ROC) of 0.63, out-performing other biomarkers, as shown in the figure below (extracted from the paper):

<img src="images/results_AUC.png" width=500/>

The remainder of this notebook is an attempt to reproduce these results using the same PPMI dataset.

## Initial setup

We first initialize the notebook cache and install dependencies:

In [1]:
import livingpark_utils

utils = livingpark_utils.LivingParkUtils()
utils.notebook_init()

removing link inputs
removing link outputs
Installing notebook dependencies (see log in install.log)... 
This notebook was run on 2022-09-22 19:43:48 UTC +0000


We fix the random seed to make sure we get the same results everytime:

In [2]:
random_seed = 1

## PPMI cohort preparation

We will build a PPMI cohort that matches the one used in the original study (Table 1) as closely as possible. Our cohort will be built directly from PPMI Study Data files so that it can be replicated and updated whenever necessary.

We define some constants that will be used in the rest of the notebook:

In [3]:
# PPMI file names
FILENAME_DEMOGRAPHICS = "Demographics.csv"
FILENAME_AGE = "Age_at_visit.csv"
FILENAME_PARTICIPANT_STATUS = "Participant_Status.csv"
FILENAME_MOCA = "Montreal_Cognitive_Assessment__MoCA_.csv"
FILENAME_UPDRS2 = "MDS_UPDRS_Part_II__Patient_Questionnaire.csv"
FILENAME_UPDRS3_CLEAN = "MDS_UPDRS_Part_III_clean.csv"
FILENAME_ADL = "Modified_Schwab___England_Activities_of_Daily_Living.csv"

# useful column names
COL_PAT_ID = "PATNO"
COL_COHORT = "COHORT_DEFINITION"
COL_VISIT_TYPE = "EVENT_ID"
COL_MRI_COMPLETE = "MRICMPLT"
COL_DATE_INFO = "INFODT"
COLS_DATE = [
    COL_DATE_INFO,
    "LAST_UPDATE",
    "ORIG_ENTRY",
]
COL_MOCA = "MCATOT"
COL_ADL = "MSEADLG"
COL_UPDRS2 = "NP2PTOT"
COL_UPDRS3 = "NP3TOT"
# PIGD: Postural Instability and Gait Disturbance score
# computed from UPDRS III measures (Stebbins et al. 2013)
COL_PIGD = "PIGD"
COLS_PIGD_COMPONENTS = [
    "NP2WALK",
    "NP2FREZ",
    "NP3FRZGT",
    "NP3GAIT",
    "NP3PSTBL",
]
COL_IMAGING_PROTOCOL = "Imaging Protocol"  # from IDA search results

# column names obtained from searching the Image and Data Archive (IDA)
# are different from those used in the PPMI csv study files
# so they need to be remapped
IDA_COLNAME_MAP = {
    "Subject ID": COL_PAT_ID,
    "Visit": COL_VISIT_TYPE,
    "Study Date": COL_DATE_INFO,
}

# codes for COHORT_DEFINITION field
STATUS_PD = "Parkinson's Disease"
STATUS_HC = "Healthy Control"

# main/validation cohorts for analysis
MAIN_COHORT = "main"
VALIDATION_COHORT = "validation"

We define some helper functions for loading and manipulating dataframes:

In [4]:
from pathlib import Path
import pandas as pd


def load_ppmi_csv(
    utils: livingpark_utils.LivingParkUtils,
    filename: str,
    from_ida_search=False,
    convert_dates: bool = True,
    alternative_dir: str = ".",
    **kwargs,
) -> pd.DataFrame:
    """
    Loads PPMI csv file as pd.DataFrame.
    Looks for file in utils.study_files_dir first, then in data_dir.
    Optionally converts date columns from str to pandas datetime type.
    """
    filepath = Path(utils.study_files_dir, filename)

    if not filepath.exists():
        filepath = Path(alternative_dir, filename)
    if not filepath.exists():
        raise FileNotFoundError(
            f"File {filename} is not in either "
            f"{utils.study_files_dir} or {alternative_dir}"
        )
    df_ppmi = pd.read_csv(filepath)

    # convert IDA search results to the same format as other PPMI study files
    if from_ida_search:
        df_ppmi = df_ppmi.rename(columns=IDA_COLNAME_MAP)
        df_ppmi[COL_PAT_ID] = pd.to_numeric(df_ppmi[COL_PAT_ID], errors="coerce")

    if convert_dates:
        df_ppmi = convert_date_cols(df_ppmi, **kwargs)
    return df_ppmi


def convert_date_cols(df: pd.DataFrame, cols: list = None):
    """
    Converts date columns from str to pandas datetime type.
    """
    if cols is None:
        cols = COLS_DATE
    for col in cols:
        try:
            df[col] = pd.to_datetime(df[col])
        except KeyError:
            continue
    return df


def filter_date(
    df: pd.DataFrame,
    max_date=None,
    min_date=None,
    dayfirst=True,
    col_date: str = COL_DATE_INFO,
) -> pd.DataFrame:
    """
    Filters a dataframe based on values in a date columns.
    """

    if type(min_date) == str:
        min_date = pd.to_datetime(min_date, dayfirst=dayfirst)
    if type(max_date) == str:
        max_date = pd.to_datetime(max_date, dayfirst=dayfirst)

    if min_date is not None:
        df = df.loc[df[col_date] >= min_date]
    if max_date is not None:
        df = df.loc[df[col_date] <= max_date]
    return df

### Study data download

We will start by downloading the PPMI Study Data files required to build our cohort: 

* Demographics
* Age at visit
* Participant status (Parkinson's disease, healthy control, etc.)
* Clinical/cognitive assessment results:
    * Montreal Cognitive Assessment (MoCA)
    * Unified Parkinson's Disease Rating Scale (UPDRS) Part II and Part III
    * Modified Schwab and England Activities of Daily Living scale

We will use the LivingPark utils library to download these files from the notebook. If files are already present in the notebook cache, they won't be downloaded again. Otherwise, you will need to enter your PPMI username and password. In case you don't have a PPMI account, you can request one [here](http://ppmi-info.org).

In [5]:
required_files = [
    FILENAME_PARTICIPANT_STATUS,
    FILENAME_DEMOGRAPHICS,
    FILENAME_AGE,
    FILENAME_MOCA,
    FILENAME_UPDRS2,
    FILENAME_ADL,
]

utils.download_ppmi_metadata(required_files)

Download skipped: No missing files!


We also download a file with information about all available T1 scans from the PPMI imaging database search tool: 

In [6]:
import ppmi_downloader

ppmi = ppmi_downloader.PPMIDownloader()
filename_t1_info = ppmi.download_3D_T1_info(destination_dir=utils.study_files_dir)

Message: element not interactable
  (Session info: headless chrome=105.0.5195.125)
Stacktrace:
0   chromedriver                        0x000000010590b788 chromedriver + 4515720
1   chromedriver                        0x000000010588f9d3 chromedriver + 4008403
2   chromedriver                        0x0000000105521feb chromedriver + 413675
3   chromedriver                        0x0000000105553a1f chromedriver + 616991
4   chromedriver                        0x0000000105552fd8 chromedriver + 614360
5   chromedriver                        0x00000001055767d2 chromedriver + 759762
6   chromedriver                        0x000000010554e075 chromedriver + 594037
7   chromedriver                        0x000000010557692e chromedriver + 760110
8   chromedriver                        0x0000000105589bd9 chromedriver + 838617
9   chromedriver                        0x0000000105576603 chromedriver + 759299
10  chromedriver                        0x000000010554c990 chromedriver + 588176
11  chromedr

### Base cohorts

The paper uses a main cohort and a validation cohort. The **main cohort** consists of subjects with 3T T1 scans and is a subset of the cohort the authors used in the 2015 paper. The **validation cohort** consists of subjects with 1.5T T1 scans.

For the **main cohort**, in the 2015 paper, the authors state that they used scans acquired from September 2013 to January 2014. However, if we restrict our search to this time period, the number of available scans is much lower than expected (38 PD patients instead of 232, 1 healthy control instead of 117). If we instead use all subjects with 3T scans from January 31st 2014 and before, we obtain sample sizes that are closer to what we expect.

For the **validation cohort**, the paper states that data were downloaded in October 2017, so we use all subjects with 1.5T scans from October 31st 2017 and before. The original validation cohort in the paper consisted of 140 PD patients.

In [7]:
import numpy as np

min_date = None
max_dates = {
    MAIN_COHORT: "31/01/2014",
    VALIDATION_COHORT: "31/10/2017",
}
T1_field_strengths = {
    MAIN_COHORT: "3.0",
    VALIDATION_COHORT: "1.5",
}
status_groups = {
    MAIN_COHORT: [STATUS_PD, STATUS_HC],
    VALIDATION_COHORT: [STATUS_PD],
}

df_t1 = load_ppmi_csv(utils, filename_t1_info, from_ida_search=True)
df_status = load_ppmi_csv(utils, FILENAME_PARTICIPANT_STATUS)

cohort_subjects = {}
for cohort_name, field_strength in T1_field_strengths.items():
    print(f"=============== {cohort_name.capitalize()} cohort ===============")
    # filter by date
    df_t1_subset = filter_date(
        df_t1, max_date=max_dates[cohort_name], min_date=min_date
    )
    # filter by field strength
    df_t1_subset = df_t1_subset.loc[
        df_t1[COL_IMAGING_PROTOCOL].str.contains(f"Field Strength={field_strength}")
    ]
    # only keep PD patients and healthy controls
    df_t1_subset = df_t1_subset.merge(df_status, on=COL_PAT_ID)
    df_t1_subset = df_t1_subset.loc[
        df_t1_subset[COL_COHORT].isin(status_groups[cohort_name])
    ]
    cohort_subjects[cohort_name] = df_t1_subset[COL_PAT_ID].drop_duplicates().tolist()
    print(
        df_status.loc[
            df_status[COL_PAT_ID].isin(cohort_subjects[cohort_name]), COL_COHORT
        ].value_counts()
    )

=============== Main cohort ===============
Parkinson's Disease    243
Healthy Control        116
Name: COHORT_DEFINITION, dtype: int64
=============== Validation cohort ===============
Parkinson's Disease    146
Name: COHORT_DEFINITION, dtype: int64
